In [1]:
# -*- coding: UTF-8 -*-    
import pandas as pd
from pandas.io.json import json_normalize
import json
import pathlib

In [5]:
# 读入数据
p = pathlib.Path('./json')
files = list(p.glob('*.json'))

dframes = []
for file in files:
    with file.open() as f:
        data_str = f.read()
        data_list = json.loads(data_str)
        df = json_normalize(data_list["list"])
        df['symbol'] = file.stem.lower()
        dframes.append(df)

res = pd.concat(dframes)
res.to_csv('account.csv', index=False)